In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from collections import defaultdict

import sys
sys.path.append('..')
from model_utils import prompt_classification

cols_output = ['final_answer_found',
               'final_model_queries', 'final_num_suffixes_checked']

In [29]:
import numpy as np
class fake_args:
    template_num_task_phrasing = 0
    max_dset_size = 200
    max_digit = 10
    seed = 1
    train_split_frac = 0.9 # we'll just test on 10%

    # these will be varied
    n_shots = 1
    task_name = 'add_two'
args = fake_args()
np.random.seed(args.seed)


In [23]:
checkpoint = 'EleutherAI/gpt-j-6B'
task_name = 'sst2_train'
d = defaultdict(list)

print('loading', checkpoint)
model = prompt_classification.create_model(checkpoint)

loading EleutherAI/gpt-j-6B


In [30]:
from model_utils import prompt_classification
import data

print('calculating accs...')
for n_shots in [1, 6]: 
    for restrict_to_valid_answers in [True, False]:
        for prompt in ['', 'manual']:    
                args.task_name = task_name
                args.n_shots = n_shots
                (dset, dset_test), check_answer_func, descr = data.get_data(
                    args, args.task_name, n_shots=args.n_shots,
                    train_split_frac=args.train_split_frac
                )
                d['checkpoint'].append(checkpoint)
                d['prompt'].append(prompt)
                d['task_name'].append(task_name)
                d['n_shots'].append(n_shots)
                d['restrict_to_valid_answers'].append(restrict_to_valid_answers)
                if prompt == 'manual':
                    prompt_actual = descr
                else:
                    prompt_actual = prompt
                d['prompt_actual'].append(prompt_actual)
                batch_size = 4
                if task_name == 'task107_splash_question_to_sql':
                    batch_size = max(1, batch_size//4)
                loss, acc = prompt_classification.test_model_on_task_with_prefix(
                    dset=dset, model=model,
                    prefix=prompt_actual,
                    multi_token=False,
                    verbose=False,
                    restrict_to_valid_answers=restrict_to_valid_answers,
                    batch_size=batch_size,
                )
                d['acc'].append(acc)

calculating accs...


In [33]:
import pandas as pd
df = pd.DataFrame.from_dict(d)
df

,acc,restrict_to_valid_answers,checkpoint,prompt,task_name,n_shots,prompt_actual
0,51.111111,True,EleutherAI/gpt-j-6B,,sst2_train,1,
1,61.666667,True,EleutherAI/gpt-j-6B,manual,sst2_train,1,"Answer ""positive"" or ""negative"" depending on t..."
2,0.000000,False,EleutherAI/gpt-j-6B,,sst2_train,1,
3,22.777778,False,EleutherAI/gpt-j-6B,manual,sst2_train,1,"Answer ""positive"" or ""negative"" depending on t..."
4,78.888889,True,EleutherAI/gpt-j-6B,,sst2_train,6,
5,73.333333,True,EleutherAI/gpt-j-6B,manual,sst2_train,6,"Answer ""positive"" or ""negative"" depending on t..."
6,78.888889,False,EleutherAI/gpt-j-6B,,sst2_train,6,
7,73.333333,False,EleutherAI/gpt-j-6B,manual,sst2_train,6,"Answer ""positive"" or ""negative"" depending on t..."
